In [1]:
import pandas as pd 
import numpy as np
import re
import sys, getopt
import csv
import os
import matplotlib.pyplot as plt
import seaborn as sns

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

os.chdir('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings')

os.getcwd()

'/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings'

In [2]:
latest_match_data = load_csv_dataset('create_dataset_1/CSVs we use often/latest_match_data_july_21_superstar_info_recalculated_columns_added.csv')
latest_match_data = latest_match_data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
#pd.set_option('display.max_columns', None)
market_values = load_csv_dataset("/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/market_value_data_confederations_added_june_18.csv")
market_values_ii = load_csv_dataset('create_dataset_1/CSVs we use often/most_updated_transfermarkt_dataset.csv')
market_values_ii['Market Value Numeric'] = pd.to_numeric(market_values_ii['Market Value'], errors='coerce')



/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_39501/1806818139.py:11: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)
/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_39501/1806818139.py:11: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


# Step 1: obtain positions (finished)

## FUNCTIONS & WORK


In [151]:
from unidecode import unidecode

#QUERY GOOGLE FUNCTIONS
import requests
import bs4
from bs4 import BeautifulSoup 

def clean_player_name(player, position, total_players):
    if position == 1:
        return player[2:-1]
    elif position == total_players:
        return player[1:-2]
    else:
        return player[1:-1]

def normalize_country_name(name):
    if name == 'DR Congo':
        return 'Democratic Republic of the Congo'
    elif name == 'USA':
        return 'United States'
    elif name == 'Congo':
        return 'Republic of the Congo'
    elif name == 'Ireland':
        return 'Republic of Ireland'
    elif name == 'Bosnia':
        return 'Bosnia-Herzegovina'
    elif name == 'Brunei':
        return 'Brunei Darussalam'
    elif name == 'US Virgin Islands':
        return 'United States Virgin Islands'
    elif name == 'Turks and Caicos Iss':
        return 'Turks and Caicos Islands'
    return name

def get_player_position(name, country_name, date, df):
    country_name = normalize_country_name(country_name)
    year = pd.to_datetime(date).year
    pos = df[(df['Name'] == name) & 
             (df['Nationality'] == country_name) & 
             (df['Season'] == year)]['Position']
    
    if pos.empty:
        player_exists = df[(df['Name'] == name) & 
                           (df['Nationality'] == country_name)]
        if not player_exists.empty:
            closest_season = player_exists.iloc[(player_exists['Season'] - year).abs().argsort()[:1]]
            pos = closest_season['Position']
    
    return pos.values[0] if not pos.empty else np.nan

def find_transfermarkt_links(html_content):
    # Regular expression patterns for transfermarkt URLs
    transfermarkt_patterns = [
        r"transfermarkt\.de",
        r"transfermarkt\.us",
        r"transfermarkt\.world"
    ]
    transfermarkt_links = []

    # Find all 'a' tags in the HTML content
    links = html_content.find_all('a', href=True)

    # Filter links that match transfermarkt patterns
    for link in links:
        for pattern in transfermarkt_patterns:
            if re.search(pattern, link['href']):
                transfermarkt_links.append(link['href'])
                break

    return transfermarkt_links


def query_google_1(input_string):
    # Replace spaces with plus signs to format the query string
    query = '+'.join(input_string.split())

    # URL for Google search
    url = f"https://www.google.com/search?q={query}"

    # Send GET request to Google
    response = requests.get(url)

    # Check if request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        # Return the parsed HTML content
        return soup
    else:
        # If request fails, print error message
        print("Failed to fetch results from Google.")
        return None
    

def extract_name_from_link(link):
        
    name_in_link = link.split('/')[3]
    name_in_link_tokens = name_in_link.split('-')
    capitalized_tokens = []

    for token in name_in_link_tokens:
        capitalized_tokens.append(token.capitalize())

    combined_string = ' '.join(capitalized_tokens)

    return combined_string


def process_google_query(input_text):
    html_result = query_google_1(input_text)
    if html_result:
        transfermarkt_links = find_transfermarkt_links(html_result)
        if transfermarkt_links:
            names_links = []
            for link in transfermarkt_links:
                if link.startswith('/url?q='):
                    link = link.split('/url?q=')[1]
                name_from_link = extract_name_from_link(link)
                names_links.append(name_from_link)
            if len(names_links) == 1:
                name = names_links[0]
                return name
            elif len(names_links) >= 2:
                name_list_matches = list(set(names_links))
                return name_list_matches
            else:
                return 0  # Nobody found
        else:
            return 0  # No links found
    else:
        return 0  # No HTML result
    
def process_google_query_new(input_text):
    html_result = query_google_1(input_text)
    if html_result:
        search_results = []
        for g in html_result.find_all('div', class_='g'):
            link = g.find('a')
            description = g.find('span', class_='aCOpRe')
            if link and description:
                search_results.append({
                    'link': link['href'],
                    'description': description.get_text()
                })
        
        # Extracting Google Knowledge Panel
        knowledge_panel = html_result.find('div', class_='kp-wholepage')
        knowledge_panel_text = knowledge_panel.get_text() if knowledge_panel else "No knowledge panel found"
        
        return search_results, knowledge_panel_text
    else:
        return None, "No HTML result"
    
def find_position(set_of_descriptions, query):
    # Define keywords and phrases in the specified order
    position_keywords = [
        "; Main position: ",
        " ➤ ",
        "; Position.",
        "; Position: ",
        " as a ",
        " as "
    ]
    

    # Define keywords for position categories
    category_keywords = {
        "Goalkeeper": ["Goalkeeper", "GK"],
        "Defender": ["DF", "Defence", "Defender"],
        "Midfielder": ["MF", "midfielder"],
        "Forward": ["FW", "attacker", "ST", "forward"]
    }

    # Initialize counters for each category
    category_counts = {key: 0 for key in category_keywords}

    # Function to check for position based on defined keywords
    def check_position(description):
        # Define the positions of each keyword and their extraction strategy
        #print('testing', description)
        if " ➤ " in description:
            parts = description.split(" ➤ ")
            if len(parts) >= 3:
                if '...' in parts[2]:
                    0==0
                else:
                    return parts[2].split(" ➤ ")[0].strip()
        elif "; Position." in description:
            return description.split("; Position.")[1].strip()
        elif "; Main position: " in description:
            start = description.index("; Main position: ") + len("; Main position: ")
            end = description.find(";", start)
            return description[start:end].strip()
        
        elif "; Position: " in description:
            start = description.index("; Position: ") + len("; Position: ")
            end = description.find(";", start)
            return description[start:end].strip()
        
        elif ' · Position:' in description:
            #print('found it', description)
            edited_description = description.split(' · Position:')[1]
            return edited_description.split(' ·')[0]
        
        elif " as a " in description:
            #print('found it', description)
            start = description.index(" as a ") + len(" as a ")
            end = description.find(".", start)
            if '...' in description[start:end]:
                #print('working', description[start:end])
                0==0
            elif '..' in description[start:end]:
                #print('working', description[start:end])
                0==0
            else:
                return description[start:end].strip()
        
        elif " as " in description:
            #print('found it', description)
            start = description.index(" as ") + len(" as ")
            end = description.find("for", start)
            if '...' in description[start:end]:
                #print('working', description[start:end])
                0==0
            elif '..' in description[start:end]:
                #print('working', description[start:end])
                0==0
            else:
                return description[start:end].strip()

        return None

    # Remove results containing "Missing: " followed by part of the query
    query_parts = query.split()
    filtered_descriptions = []
    for description in set_of_descriptions:
        if not any(f"Missing: {part}" in description for part in query_parts):
            filtered_descriptions.append(description)

    # Remove results that are exactly equal to or contained within the removed results
    final_descriptions = []
    for description in filtered_descriptions:
        if not any(description in removed for removed in set_of_descriptions if removed not in filtered_descriptions):
            #print('adding to second set', description)
            final_descriptions.append(description)

    # First, look for descriptions with the "➤" character
    for description in final_descriptions:
        if " ➤ " in description:
            position = check_position(description)
            if position:
                return position

    # If no position found, check other keywords
    for description in final_descriptions:
        #print('checking second set', description)
        position = check_position(description)
        if position:
            return position
        
        # Count occurrences of position keywords
        for category, keywords in category_keywords.items():
            if any(keyword in description for keyword in keywords):
                category_counts[category] += 1

    # Determine the most frequent position category
    most_frequent_category = max(category_counts, key=category_counts.get)
    
    return most_frequent_category

def look_up_player_position(player, natl, year):
    input_text_1 = f"{player} {natl} footballer {year}"
    soup1 = query_google_1(input_text_1)
    # Find all search result containers
    set_of_descriptions = []
    for result in soup1.find_all('div', class_='BNeawe s3v9rd AP7Wnd'):
        # Extract the description
        description = result.get_text()
        # Find the parent container for the title
        parent = result.find_parent('div', class_='kCrYT')
        if parent:
            # Extract the title
            title_div = parent.find('div', class_='BNeawe vvjwJb AP7Wnd')
            title = title_div.get_text() if title_div else "No Title"
            #print(f"Title: {title}")
            set_of_descriptions.append(description)
            # print(f"Description: {description}\n")

    #print(f'for {player} {natl} {year}, set of descriptions is {set_of_descriptions}')
    # Example usage
    position = find_position(set_of_descriptions, input_text_1)
    if 'for' in position:
        position = position.split('for')[0]
    elif ' ; ' in position:
        position = position.split(' ; ')[0]
    elif '.Missing' in position:
        position = position.split('.Missing')[0]
    
    # print("Position:", position)
    return position
    
def get_player_position_final(player, nationality, date, competition, market_values, market_values_ii, year):
    position = get_player_position(player, nationality, date, market_values)
    if pd.isna(position):
        # Accent removed
        position_accent_removed = get_player_position(unidecode(player), nationality, date, market_values)
        if pd.isna(position_accent_removed):
            # Other data
            position_other_data = get_player_position(player, nationality, date, market_values_ii)
            if pd.isna(position_other_data):
                if competition == 'Olympics':
                    # Check if player has played for another country (Olympics only)
                    if len(market_values[(market_values['Name'] == player) & (market_values['Nationality'] != nationality)]) > 0:
                        other_nationality = market_values[(market_values['Name'] == player) & (market_values['Nationality'] != nationality)]['Nationality'].unique()[0]
                        position_other_nationality = get_player_position(player, other_nationality, date, market_values)
                        return position_other_nationality
                    else:
                        print('looking up player position', player, nationality, year)
                        position_online = look_up_player_position(player, nationality, year)
                        return position_online
                else:
                    if player == 'Oscar':
                        player = 'Oscar Dorley'
                    print('looking up player position', player, nationality, year)
                    position_online = look_up_player_position(player, nationality, year)
                    return position_online
            else:
                return position_other_data
        else:
            return position_accent_removed
    else:
        return position

In [319]:
#ADDING POSITIONS TO DATA 

# latest_match_data['Status'] = 'Not done'
# latest_match_data['Team.1.Players_Positions'] = ''
# latest_match_data['Team.2.Players_Positions'] = ''


# upper_index = 1700
# lower_index = 1500
upper_index += 19
lower_index += 200
for index, row in latest_match_data.iterrows():
    if lower_index <= index <= upper_index:
        print(index, '*')
        if row['Competition'] == 'Olympics':
            latest_match_data.at[index, 'Status'] = 'Done'
            latest_match_data.at[index, 'Team.1.Players_Positions'] = ''
            latest_match_data.at[index, 'Team.2.Players_Positions'] = ''
        elif row['Status'] == 'Not done':
            team_1_list_final = []
            team_2_list_final = []

            year = row['Season']
            team_1_list = row['Team.1.Players']
            competition = row['Competition']
            team_2_list = row['Team.2.Players']
            team_1_list = eval(team_1_list)
            team_2_list = eval(team_2_list)
            team_1_nationality = row['Team.1.Name']
            team_2_nationality = row['Team.2.Name']

            if team_1_nationality == 'Türkiye':
                team_1_nationality = 'Turkey'
            elif team_2_nationality == 'Türkiye':
                team_2_nationality = 'Turkey'

            for player in team_1_list:
                position = get_player_position_final(player, team_1_nationality, row['Date'], competition, market_values, market_values_ii, year)
                team_1_list_final.append(position)

            for player in team_2_list:
                position = get_player_position_final(player, team_2_nationality, row['Date'], competition, market_values, market_values_ii, year)
                team_2_list_final.append(position)

            # print(row['Team.1.Players'], team_1_list_final)
            # print(row['Team.2.Players'], team_2_list_final)
            latest_match_data.at[index, 'Team.1.Players_Positions'] = team_1_list_final
            latest_match_data.at[index, 'Team.2.Players_Positions'] = team_2_list_final
            latest_match_data.at[index, 'Status'] = 'Done'
        else:
            if row['Team.1.Players_Positions'] == '':
                0==0
            else:
                latest_match_data.at[index, 'Status'] = 'Done'       

5300 *
5301 *
5302 *
5303 *
5304 *
5305 *
5306 *
5307 *
5308 *
5309 *
looking up player position Cardoso São Tomé and Príncipe 2022
5310 *
5311 *
5312 *
5313 *
5314 *
looking up player position Rethabile Mokokoane Lesotho 2022
5315 *
5316 *
5317 *
5318 *
looking up player position Cardoso São Tomé and Príncipe 2022


In [ ]:
#checking stuff for manual fixes
for index, row in latest_match_data.iterrows():
    if index == 1217:
        year = row['Season']
        team_1_list = row['Team.1.Players']
        competition = row['Competition']
        team_2_list = row['Team.2.Players']
        team_1_list = eval(team_1_list)
        team_2_list = eval(team_2_list)
        team_1_nationality = row['Team.1.Name']
        team_2_nationality = row['Team.2.Name']

        for player in team_2_list:
            #if player == 'Foster Namwera':
                position = get_player_position_final(player, team_2_nationality, row['Date'], competition, market_values, market_values_ii, year)
                print(player, position)
            #elif player == 'Wonderful Jeremani':
                # position = get_player_position_final(player, team_1_nationality, row['Date'], competition, market_values, market_values_ii, year)
                # print(player, position)

In [377]:
def find_correct_position(input_position, index):
    if input_position == 'winger':
        return 'Left Winger'
    elif input_position == 'right back':
        return 'Right-Back'
    elif input_position == 'Centre Forward (CF)':
        return 'Centre-Forward'
    elif input_position == 'left-back':
        return 'Left-Back'
    elif input_position == 'goalkeeper ':
        return 'Goalkeeper'
    elif input_position == 'goalkeeper':
        return 'Goalkeeper'
    elif input_position == 'substitute':
        return 'Centre-Forward'
    elif input_position == ' Central Midfield.':
        return 'Central Midfield'
    elif input_position == 'centre-back ':
        return 'Centre-Back'
    elif input_position == 'Sweeper':
        return 'Centre-Back'
    elif input_position == 'central defender ':
        return 'Centre-Back'
    elif input_position == 'left back ':
        return 'Left-Back'
    elif input_position == 'defender ':
        if (index >= 3258 and index <= 3271) or (index >= 3445 and index <= 3466) or index == 3472 or index == 3490:
            return 'Centre-Back'
        else:
            return 'Centre-Back'
    elif input_position == 'midfield':
        return 'Central Midfield'
    elif input_position == 'an attacking midfielder':
        return 'Attacking Midfield'
    elif input_position == 'midfielder ':
        return 'Central Midfield'
    elif input_position == 'right winger ':
        return 'Right Winger'
    elif input_position == 'left winger ':
        return 'Left Winger'
    elif input_position == 'second striker ':
        return 'Second Striker'
    elif input_position == 'Right Winger. Current ... Sep 19, 2019, APOEL Nicosia 3:4 · F91 Dudelange · Thomas Doll, 22 years 03 ..':
        return 'Right Winger'
    elif 'Midfielder.' in input_position:
        return 'Central Midfield'
    elif 'Goalkeeper.' in input_position:
        return 'Goalkeeper'
    elif input_position == 'striker':
        return 'Centre-Forward'
    elif input_position == 'striker ':
        return 'Centre-Forward'
    elif input_position == 'defensive midfielder ':
        return 'Defensive Midfield'
    elif input_position == 'Left Back (LB)':
        return 'Left-Back'
    elif input_position == 'Midfield':
        return 'Central Midfield'
    elif 'Defender.' in input_position:
        return 'Defender'
    elif input_position == 'Defender (D)':
        return 'Left-Back'
    elif input_position == 'left-back ':
        return 'Left-Back'
    elif 'Right Winger.' in input_position:
        return 'Right Winger'
    else:
        return input_position



                                                            # if you see winger, that's Left Winger
                                                            # if you see right back, that becomes Right-Back
                                                            # if you see Centre Forward (CF), that becomes Centre-Forward   
                                                            # if you see left-back, that becomes Left-Back
                                                            # if you see goalkeeper that becomes Goalkeeper 
                                                            # if you see substitute and the index is 2110 2112 or 2114 that becomes Centre-Forward
                                                            #if you see ' Central Midfield.' that becomes Central Midfield
                                                            #if you see 'centre-back ' that becomes Centre-Back
                                                            #if you see Sweeper that becomes Center-Back

                                                            #if you see 'central defender ' that becomes Centre-Back
                                                            #if you see 'left back ' that becomes Left-Back
                                                            #if you see 'defender ' and index is between 3258 and 3271 or between 3445 and 3466, or 3472, or 3490 that becomes Center back 
                                            #if you see 'midfield' that becomes Central Midfield
                                            #if you see 'an attacking midfielder' that becomes Attacking Midfield
                                            #if you see 'midfielder ' that becomes Central Midfield
                                            #if you see 'right winger ' that becomes Right Winger
                                            #if you see 'left winger ' that becomes Left Winger
                                            #if you see 'second striker ' that becomes Second Striker
                            #if you see 'Right Winger. Current ... Sep 19, 2019, APOEL Nicosia 3:4 · F91 Dudelange · Thomas Doll, 22 years 03 ..' that becomes Right Winger
                            #if you see 'Midfielder.' with stuff after, that becomes Central Midfield
                            #if you see 'Goalkeeper.' with stuff after, that becomes Goalkeeper
                            #if you see striker, that becomes Centre-Forward
                    #if you see 'defensive midfielder ' that becomes Defensive Midfield
                    #if you see Left Back (LB) that becomes Left-Back
                    #if you see Midfield that becomes Central Midfield
                    #if you see 'Defender.' with stuff after, just take Defender
                    #if you see Defender (D) that becomes Left-Back

In [381]:
row['Team.1.Players_Positions']

['Goalkeeper',
 'Centre-Back',
 'Centre-Back',
 'Right Winger',
 'Defensive Midfield',
 'Midfielder',
 'Attacking Midfield',
 'Right Winger',
 'Left Winger',
 'Centre-Forward',
 'Right Winger']

In [387]:
latest_match_data.to_csv('latest_match_data_july_25_POSITIONS_ADDED.csv')

In [391]:
for index, row in latest_match_data.iterrows():
    if 0 <= index <= 5319:
        if row['Competition'] != 'Olympics':
            for i in range(0, 11):
                position = row['Team.1.Players_Positions'][i]
            if position not in ['Goalkeeper', 'Defender', 'Midfielder', 'Striker', 'Left Midfield', 'Right Midfield', 'Left Winger', 'Right Winger', 'Forward', 'Centre-Forward', 'Centre-Back', 'Left-Back', 'Right-Back', 'Central Midfield', 'Attacking Midfield', 'Second Striker', 'Defensive Midfield']:
                #print('not in')
                position_to_fix_player = eval(row['Team.1.Players'])[i]
                print('TO FIX team 1', index, position, position_to_fix_player)   
                fixed_position = find_correct_position(position, index)
                
                print('FIXED', index, fixed_position, position_to_fix_player)
                print('we want to change: ', latest_match_data.at[index, 'Team.1.Players_Positions'][i])
                latest_match_data.at[index, 'Team.1.Players_Positions'][i] = fixed_position
                print('now we have: ', latest_match_data.at[index, 'Team.1.Players_Positions'][i])
                print('')
            if position == 'Attacker':
                print('yeah')
                

            for i in range(0, 11):
                position = row['Team.2.Players_Positions'][i]
                if position not in ['Goalkeeper', 'Defender', 'Midfielder', 'Striker', 'Left Midfield', 'Right Midfield', 'Left Winger', 'Right Winger', 'Forward', 'Centre-Forward', 'Centre-Back', 'Left-Back', 'Right-Back', 'Central Midfield', 'Attacking Midfield', 'Second Striker', 'Defensive Midfield']:
                    position_to_fix_player = eval(row['Team.2.Players'])[i]
                    print('TO FIX team 2', index, position, position_to_fix_player)   
                    fixed_position = find_correct_position(position, index)
                    
                    print('FIXED', index, fixed_position, position_to_fix_player)
                    print('we want to change: ', latest_match_data.at[index, 'Team.2.Players_Positions'][i])# = fixed_position
                    latest_match_data.at[index, 'Team.2.Players_Positions'][i] = fixed_position
                    print('now we have: ', latest_match_data.at[index, 'Team.2.Players_Positions'][i])
                    print('')
                if position == 'Attacker':
                    print('yeah')
                    

In [549]:
latest_match_data_updated = load_csv_dataset('create_dataset_1/CSVs we use often/latest_match_data_july_25_POSITIONS_ADDED.csv')

In [550]:
latest_match_data_updated.at[40, 'Team.2.Players_Positions'] = "['Goalkeeper', 'Centre-Back', 'Centre-Back', 'Left-Back', 'Centre-Back', 'Right-Back', 'Defensive Midfield', 'Defensive Midfield', 'Central Midfield', 'Central Midfield', 'Centre-Forward']"

latest_match_data_updated.at[260, 'Team.2.Players_Positions'] = "['Goalkeeper', 'Centre-Back', 'Centre-Back', 'Centre-Back', 'Central Midfield', 'Central Midfield', 'Right Winger', 'Right-Back', 'Right Winger', 'Second Striker', 'Centre-Forward']"

latest_match_data_updated.at[272, 'Team.1.Players_Positions'] = "['Goalkeeper', 'Centre-Back', 'Centre-Back', 'Left-Back', 'Centre-Back', 'Defensive Midfield', 'Central Midfield', 'Right Winger', 'Right Winger', 'Second Striker', 'Centre-Forward']"

latest_match_data_updated.at[278, 'Team.1.Players_Positions'] = "['Goalkeeper', 'Centre-Back', 'Centre-Back', 'Centre-Back', 'Right Winger', 'Central Midfield', 'Right Winger', 'Central Midfield', 'Left-Back', 'Right Winger', 'Centre-Forward']"

latest_match_data_updated.at[283, 'Team.1.Players_Positions'] = "['Goalkeeper', 'Centre-Back', 'Centre-Back', 'Right-Back', 'Centre-Back', 'Central Midfield', 'Central Midfield', 'Right Winger', 'Right-Back', 'Second Striker', 'Centre-Forward']"

latest_match_data_updated.at[288, 'Team.1.Players_Positions'] = "['Goalkeeper', 'Centre-Back', 'Centre-Back', 'Right-Back', 'Left-Back', 'Central Midfield', 'Central Midfield', 'Right Winger', 'Right Winger', 'Centre-Forward', 'Centre-Forward']"

latest_match_data_updated.at[310, 'Team.2.Players_Positions'] = "['Goalkeeper', 'Centre-Back', 'Centre-Back', 'Left-Back', 'Right-Back', 'Centre-Back', 'Attacking Midfield', 'Attacking Midfield', 'Defensive Midfield', 'Centre-Forward', 'Centre-Forward']"

latest_match_data_updated.at[1772, 'Team.1.Players_Positions'] = "['Goalkeeper', 'Centre-Back', 'Centre-Back', 'Centre-Back', 'Left-Back', 'Right-Back', 'Central Midfield', 'Right Winger', 'Defensive Midfield', 'Centre-Forward', 'Centre-Forward']"

latest_match_data_updated.at[1774, 'Team.2.Players_Positions'] = "['Goalkeeper', 'Centre-Back', 'Centre-Back', 'Right-Back', 'Left-Back', 'Centre-Back', 'Defensive Midfield', 'Central Midfield', 'Right Winger', 'Centre-Forward', 'Centre-Forward']"

latest_match_data_updated.at[3950, 'Team.1.Players_Positions'] = "['Goalkeeper', 'Defender', 'Centre-Back', 'Left-Back', 'Right-Back', 'Centre-Back', 'Central Midfield', 'Attacking Midfield', 'Centre-Forward', 'Centre-Forward', 'Right Midfield']"

latest_match_data_updated.at[3955, 'Team.2.Players_Positions'] = "['Goalkeeper', 'Defender', 'Centre-Back', 'Centre-Back', 'Right-Back', 'Central Midfield', 'Right Winger', 'Centre-Forward', 'Centre-Forward', 'Centre-Forward', 'Right Midfield']"

latest_match_data_updated.at[4151, 'Team.1.Players_Positions'] = "['Goalkeeper', 'Centre-Back', 'Defensive Midfield', 'Left Winger', 'Right-Back', 'Central Midfield', 'Defensive Midfield', 'Defensive Midfield', 'Left Winger', 'Centre-Forward', 'Right Winger']"

latest_match_data_updated.at[4164, 'Team.2.Players_Positions'] = "['Goalkeeper', 'Centre-Back', 'Defensive Midfield', 'Defensive Midfield', 'Right-Back', 'Central Midfield', 'Right Winger', 'Left Winger', 'Central Midfield', 'Centre-Forward', 'Right Winger']"

latest_match_data_updated.at[4295, 'Team.2.Players_Positions'] = "['Goalkeeper', 'Centre-Back', 'Centre-Back', 'Centre-Back', 'Right-Back', 'Right-Back', 'Defensive Midfield', 'Central Midfield', 'Defensive Midfield', 'Right Winger', 'Centre-Forward']"

latest_match_data_updated.at[4473, 'Team.2.Players_Positions'] = "['Goalkeeper', 'Centre-Back', 'Central Midfield', 'Right-Back', 'Right-Back', 'Defensive Midfield', 'Defensive Midfield', 'Right-Back', 'Left Winger', 'Attacking Midfield', 'Second Striker']"

latest_match_data_updated.at[4596, 'Team.2.Players_Positions'] = "['Goalkeeper', 'Defender', 'Right Winger', 'Right Winger', 'Striker', 'Central Midfield', 'Defender', 'Midfielder', 'Central Midfield', 'Defender', 'Midfielder']"

latest_match_data_updated.at[4837, 'Team.2.Players_Positions'] = "['Goalkeeper', 'Centre-Back', 'Centre-Back', 'Left-Back', 'Right-Back', 'Right Winger', 'Attacking Midfield', 'Attacking Midfield', 'Striker', 'Centre-Forward', 'Right Midfield']"

latest_match_data_updated.at[4839, 'Team.1.Players_Positions'] = "['Goalkeeper', 'Centre-Back', 'Centre-Back', 'Right-Back', 'Central Midfield', 'Attacking Midfield', 'Left Winger', 'Right Winger', 'Centre-Forward', 'Centre-Forward', 'Right Midfield']"

latest_match_data_updated.at[4972, 'Team.2.Players_Positions'] = "['Goalkeeper', 'Centre-Back', 'Centre-Back', 'Left-Back', 'Defensive Midfield', 'Defensive Midfield', 'Central Midfield', 'Left Winger', 'Centre-Forward', 'Centre-Forward', 'Centre-Forward']"

latest_match_data_updated.at[3725, 'Team.1.Players_Positions'] = "['Goalkeeper', 'Centre-Back', 'Centre-Back', 'Defensive Midfield', 'Defensive Midfield', 'Defensive Midfield', 'Central Midfield', 'Left Winger', 'Left Winger', 'Centre-Forward', 'Centre-Forward']"


In [552]:
for index, row in latest_match_data_updated.iterrows():
    if row['Competition'] != 'Olympics':
        team_1_positions = row['Team.1.Players_Positions']
        team_2_positions = row['Team.2.Players_Positions']
        for i in range(0, 11):
            if team_1_positions[i] == 'Attacker':
                print(index, 'yeah', team_1_positions[i], eval(row['Team.1.Players'])[i], row['Competition'])
                fixed_position = 'Forward'
                latest_match_data.at[index, 'Team.1.Players_Positions'][i] = fixed_position
        if len(eval(team_1_positions)) != 11:
            print(index, 'team 1 not 11')
        elif len(eval(team_2_positions)) != 11:
            print(index, 'team 2 is not 11')

        if eval(team_1_positions).count('Goalkeeper') != 1:
            print(index, 'team 1 not 1 goalkeeper', eval(team_1_positions).count('Goalkeeper'), row['Team.1.Name'])
        elif eval(team_2_positions).count('Goalkeeper') != 1:
            print(index, 'team 2 not 1 goalkeeper', eval(team_2_positions).count('Goalkeeper'), row['Team.2.Name'])
latest_match_data_updated.to_csv('latest_match_data_july_25_positions_added.csv')

4783 team 1 not 1 goalkeeper 0 Eswatini


# Step 2: % of max value XI score - FINDING THE MOST VALUABLE PERSON AT EACH POSITION

In [1571]:
# latest_match_data = load_csv_dataset('create_dataset_1/CSVs we use often/latest_match_data_july_25_positions_added.csv')
latest_match_data = load_csv_dataset('create_dataset_1/CSVs we use often/latest_match_data_july_25_positions_FIXED_HOME_AWAY_62_ROWS_FIXED.csv')#latest_match_data_july_25_positions_FIXED
latest_match_data['is_draw'] = np.where(latest_match_data['Score.Difference'] == 0, 1, 0)
# latest_match_data.to_csv('latest_match_data_july_25_positions_FIXED.csv')

In [1572]:
# Identify rows where the 'Tag' column is 'Y'
tagged_rows = latest_match_data[latest_match_data['Tag'] == 'Y']

# Apply the logic to these rows
for index, row in tagged_rows.iterrows():
    # Multiply the score difference by -1
    latest_match_data.at[index, 'Score.Difference'] *= -1
    
    # Swap MV_Prediction_Correct and MV_Prediction_Correct_inverse
    mv_prediction_correct = latest_match_data.at[index, 'MV_Prediction_Correct']
    mv_prediction_correct_inverse = latest_match_data.at[index, 'MV_Prediction_Correct_inverse']
    latest_match_data.at[index, 'MV_Prediction_Correct'] = mv_prediction_correct_inverse
    latest_match_data.at[index, 'MV_Prediction_Correct_inverse'] = mv_prediction_correct
    
    # Swap rank_prediction_correct and Rank_Prediction_Correct_inverse
    rank_prediction_correct = latest_match_data.at[index, 'rank_prediction_correct']
    rank_prediction_correct_inverse = latest_match_data.at[index, 'Rank_Prediction_Correct_inverse']
    latest_match_data.at[index, 'rank_prediction_correct'] = rank_prediction_correct_inverse
    latest_match_data.at[index, 'Rank_Prediction_Correct_inverse'] = rank_prediction_correct


In [1573]:



for index, row in latest_match_data.iterrows():
    if row['Competition'] != 'Olympics':
        for i in range(0, 11):
            if eval(row['Team.1.Players_Positions'])[i] == 'Attacker':
                print(index, 'yeah', eval(row['Team.1.Players_Positions']), row['Competition'])
                test_thing = eval(row['Team.1.Players_Positions'])
                print(test_thing[i])
                test_thing[i] = 'Forward'
                latest_match_data.at[index, 'Team.1.Players_Positions'] = str(test_thing)
            elif eval(row['Team.1.Players_Positions'])[i] == 'midfielder ':
                print(index, 'yeah', eval(row['Team.1.Players_Positions'])[i])
                test_thing = eval(row['Team.1.Players_Positions'])
                test_thing[i] = 'Midfielder'
                latest_match_data.at[index, 'Team.1.Players_Positions'] = str(test_thing)
            elif eval(row['Team.1.Players_Positions'])[i] == 'defender ':
                print(index, 'yeah', eval(row['Team.1.Players_Positions'])[i])
                test_thing = eval(row['Team.1.Players_Positions'])
                test_thing[i] = 'Defender'
                latest_match_data.at[index, 'Team.1.Players_Positions'] = str(test_thing)
            elif eval(row['Team.1.Players_Positions'])[i] == 'Defender (D)':
                print(index, 'yeah', eval(row['Team.1.Players_Positions'])[i])
                test_thing = eval(row['Team.1.Players_Positions'])
                test_thing[i] = 'Defender'
                latest_match_data.at[index, 'Team.1.Players_Positions'] = str(test_thing)
            elif eval(row['Team.1.Players_Positions'])[i] == 'right back':
                print(index, 'yeah', eval(row['Team.1.Players_Positions'])[i])
                test_thing = eval(row['Team.1.Players_Positions'])
                test_thing[i] = 'Right-Back'
                latest_match_data.at[index, 'Team.1.Players_Positions'] = str(test_thing)
            elif eval(row['Team.1.Players_Positions'])[i] == 'winger':
                print(index, 'yeah', eval(row['Team.1.Players_Positions'])[i])
                test_thing = eval(row['Team.1.Players_Positions'])
                test_thing[i] = 'Right Winger'
                latest_match_data.at[index, 'Team.1.Players_Positions'] = str(test_thing)
            elif eval(row['Team.1.Players_Positions'])[i] == 'Midfielder. Helton\xa0..':
                print(index, 'yeah', eval(row['Team.1.Players_Positions'])[i])
                test_thing = eval(row['Team.1.Players_Positions'])
                test_thing[i] = 'Midfielder'
                latest_match_data.at[index, 'Team.1.Players_Positions'] = str(test_thing)

unique_positions = []
for index, row in latest_match_data.iterrows():
    for i in range(0, 11):
        if row['Competition'] != 'Olympics':
            if eval(row['Team.1.Players_Positions'])[i] not in unique_positions:
                unique_positions.append(eval(row['Team.1.Players_Positions'])[i])
            if eval(row['Team.2.Players_Positions'])[i] not in unique_positions:
                unique_positions.append(eval(row['Team.2.Players_Positions'])[i])
unique_positions
#stuff to change 

#if it's 'midfielder ' change it to Midfielder - DONE
#if its 'defender ' change it to Defender - DONE
#if it's 'Defender (D)' change it to Defender - DONE
#if it's 'right back' change it to Right-Back - DONE
#if it's 'winger' change it to Right Winger - 
#if it's 'Midfielder. Helton\xa0..' change it to Midfielder - 
#if it's Attacker change it to Forward - DONE

['Goalkeeper',
 'Centre-Back',
 'Defender',
 'Central Midfield',
 'Right-Back',
 'Defensive Midfield',
 'Attacking Midfield',
 'Midfielder',
 'Left Winger',
 'Centre-Forward',
 'Left-Back',
 'Left Midfield',
 'Right Winger',
 'Second Striker',
 'Right Midfield',
 'Forward',
 'Striker',
 '',
 'transfers']

In [1574]:
def normalize_country_name(name):
    if name == 'DR Congo':
        return 'Democratic Republic of the Congo'
    elif name == 'USA':
        return 'United States'
    elif name == 'Congo':
        return 'Republic of the Congo'
    elif name == 'Ireland':
        return 'Republic of Ireland'
    elif name == 'Bosnia':
        return 'Bosnia-Herzegovina'
    elif name == 'Brunei':
        return 'Brunei Darussalam'
    elif name == 'US Virgin Islands':
        return 'United States Virgin Islands'
    elif name == 'Turks and Caicos Iss':
        return 'Turks and Caicos Islands'
    return name

def normalize_country_name_ii(name):
    #if you are searching market_values_ii it's DR Congo and Congo not the long names. 
    if name == 'USA':
        return 'United States'
    elif name == 'Ireland':
        return 'Republic of Ireland'
    elif name == 'Bosnia':
        return 'Bosnia-Herzegovina'
    elif name == 'Brunei':
        return 'Brunei Darussalam'
    elif name == 'US Virgin Islands':
        return 'United States Virgin Islands'
    elif name == 'Turks and Caicos Iss':
        return 'Turks and Caicos Islands'
    return name

In [1575]:
# Define the position search sets
position_search_sets = {
    'Left-Back': ['Left-Back'],
    'Centre-Back': ['Centre-Back', 'Defender'],
    'Right-Back': ['Right-Back'],
    'Defender': ['Left-Back', 'Centre-Back', 'Right-Back'],
    'Midfielder': ['Central Midfield', 'Midfielder'],
    'Defensive Midfield': ['Defensive Midfield', 'Midfielder'],
    'Attacking Midfield': ['Attacking Midfield', 'Midfielder'],
    'Central Midfield': ['Central Midfield', 'Midfielder'],
    'Left Midfield': ['Left Midfield'],
    'Right Midfield': ['Right Midfield'],
    'Striker': ['Striker', 'Second Striker', 'Forward', 'Centre-Forward'],
    'Second Striker': ['Striker', 'Second Striker', 'Forward', 'Centre-Forward'],
    'Centre-Forward': ['Striker', 'Second Striker', 'Forward', 'Centre-Forward'],
    'Left Winger': ['Left Winger'],
    'Right Winger': ['Right Winger'],
    'Forward': ['Forward', 'Left Winger', 'Right Winger', 'Striker', 'Second Striker', 'Centre-Forward']
}

def get_search_positions(position):
    return position_search_sets.get(position, [position])

In [1458]:
#changes - DONE


#refactor that bit inside the for loop - DONE 
#add the actual RETURN STATEMENTS - DONE
#make the things we return actually go into the columns 
#add the duplicate name checker  - DONE
#add to the duplicate name checker - IF THE RESULTING QUOTIENT IS ABOVE 100, PRINT OUT player names and values - DONE

#add to the duplicate name checker -  DONE

#make sure the math is correct - DONE
#add a column that indicates if they've made a change - DONE


#add a checker that makes sure the lineup isn't all empty before proceedng,. -  DONE 
#add the checks for Honduras 2021 - DONE
#add the individual nation checks in notes - DONE

In [1630]:
def get_player_position(player, position, market_value, team_name, season, index, market_values, market_values_ii, list_of_names, teammates_in_XI):
    if list_of_names == []:
        list_of_names = ['']
    search_positions = get_search_positions(position)

    if team_name == 'Türkiye' and season > 2012:
        team_name = 'Turkey'

    data_subset_position = market_values[
        (market_values['Nationality'] == normalize_country_name(team_name)) &
        (market_values['Season'] == season) &
        (market_values['Position'].isin(search_positions)) & 
        (~market_values['Name'].isin(list_of_names)) & 
        (~market_values['Name'].isin(teammates_in_XI))
    ]

    data_subset_position_ii = market_values_ii[
        (market_values_ii['Nationality'] == normalize_country_name_ii(team_name)) &
        (market_values_ii['Season'] == season) &
        (market_values_ii['Position'].isin(search_positions)) & 
        (~market_values_ii['Name'].isin(list_of_names)) & 
        (~market_values_ii['Name'].isin(teammates_in_XI))
    ]

    if len(data_subset_position) == 0 and len(data_subset_position_ii) == 0:
        #print('we here', position, player)
        if position == 'Central Midfield':
            search_positions_expanded = ['Attacking Midfield', 'Defensive Midfield', 'Midfielder']
        elif position == 'Attacking Midfield':
            search_positions_expanded = ['Central Midfield', 'Midfielder']
        elif position == 'Defensive Midfield':
            search_positions_expanded = ['Central Midfield', 'Midfielder']
        else:
            search_positions_expanded = [position]
        data_subset_position_expanded = market_values[
            (market_values['Nationality'] == normalize_country_name(team_name)) &
            (market_values['Season'] == season) &
            (market_values['Position'].isin(search_positions_expanded)) & 
            (~market_values['Name'].isin(list_of_names)) & 
            (~market_values['Name'].isin(teammates_in_XI))
        ]

        data_subset_position_expanded_ii = market_values_ii[
            (market_values_ii['Nationality'] == normalize_country_name_ii(team_name)) &
            (market_values_ii['Season'] == season) &
            (market_values_ii['Position'].isin(search_positions_expanded)) & 
            (~market_values_ii['Name'].isin(list_of_names)) & 
            (~market_values_ii['Name'].isin(teammates_in_XI))
        ]

        if len(data_subset_position_expanded) == 0 and len(data_subset_position_expanded_ii) == 0:
            print(index, '- no data with expanded search', player, position, market_value, team_name, season)
            if market_value == 0:
                max_value_name, max_value, max_value_score = player, market_value, 0
            else:
                max_value_name, max_value, max_value_score = player, market_value, 1
        else:
            expanded_data_big = pd.concat([data_subset_position_expanded, data_subset_position_expanded_ii], ignore_index=True)
            if len(expanded_data_big['Name'].unique()) == 1: #there is only 1 player in this list 
                if expanded_data_big['Name'].unique()[0] == player: #if its the input player
                    max_value_name, max_value, max_value_score = player, market_value, 1
                else:
                    max_value = expanded_data_big['Market Value Numeric'].max()
                    if max_value == 0:
                        if market_value == 0:
                            max_value_name, max_value, max_value_score = player, market_value, 0
                        else:
                            max_value_name, max_value, max_value_score = player, market_value, 100
                    else:
                        max_value_name = expanded_data_big[expanded_data_big['Market Value Numeric'] == max_value]['Name'].values[0]
                        max_value_score = 100 * (market_value / max_value)
            else:
                max_value = expanded_data_big['Market Value Numeric'].max()
                if max_value == 0:
                    if market_value == 0:
                        max_value_name, max_value, max_value_score = player, market_value, 0
                    else:
                        max_value_name, max_value, max_value_score = player, market_value, 100
                else:
                    max_value_name = expanded_data_big[expanded_data_big['Market Value Numeric'] == max_value]['Name'].values[0]
                    max_value_score = 100 * (market_value / max_value)

    else:
        position_data_big = pd.concat([data_subset_position, data_subset_position_ii], ignore_index=True)
        max_value = position_data_big['Market Value Numeric'].max()
        if max_value == 0:
            if market_value == 0:
                max_value_name, max_value, max_value_score = player, market_value, 0
            else:
                max_value_name, max_value, max_value_score = player, market_value, 100
        elif pd.isna(max_value):
            if market_value == 0:
                max_value_name, max_value, max_value_score = player, market_value, 0
            else:
                max_value_name, max_value, max_value_score = player, market_value, 100
        else:
            #print(max_value, player, position, market_value, team_name, season)
            #print(position_data_big)
            #print(pd.isna(max_value))
            max_value_name = position_data_big[position_data_big['Market Value Numeric'] == max_value]['Name'].values[0]
            max_value_score = 100 * (market_value / max_value)

    if max_value_score > 100:
        max_value_score = 100
        print('max value score is greater than 100', player, position, market_value, max_value_name, max_value, max_value_score)
        #print('squad index was: ', index)
        print('list of used players is ', list_of_names)

    data_that_year = market_values[(market_values['Name'] == player) & (market_values['Season'] == season)]
    if len(data_that_year) == 0:
        player_in_data_that_year = False
    else:
        player_in_data_that_year = True

    return max_value_name, max_value, max_value_score, player_in_data_that_year

In [1577]:
africa_countries_list = ['DR Congo', 'Madagascar', 'Mali', 'Equatorial Guinea', 'Morocco',
       'Sudan', 'Namibia', 'Zimbabwe', 'Ivory Coast', 'Tanzania',
       'Zambia', 'Lesotho', 'Burkina Faso', 'Congo', 'Togo', 'Uganda',
       'Liberia', 'Gabon', 'Angola', 'Egypt', 'South Africa', 'Nigeria',
       'Algeria', 'Cameroon', 'Ghana', 'Tunisia', 'Senegal', 'Ethiopia',
       'Botswana', 'Central African Republic', 'Cape Verde',
       'Sierra Leone', 'The Gambia', 'Niger', 'Malawi', 'Kenya',
       'Mozambique', 'Guinea', 'Rwanda', 'Benin', 'Libya', 'Chad',
       'Seychelles', 'Eritrea', 'São Tomé and Príncipe', 'Eswatini',
       'Guinea-Bissau', 'Comoros', 'Somalia', 'Burundi', 'Mauritania',
       'Mauritius', 'South Sudan', 'Djibouti']

skip_conditions = {
    'DR Congo': [2013],
    'Sudan': [2012, 2013],
    'The Gambia': [2012, 2013],
    'Sierra Leone': [2012, 2013],
    'Tanzania': [2012, 2013],
    'Liberia': [2012, 2013],
    'Botswana': [2012, 2013],
    'Benin': [2012, 2013],
    'Kenya': [2013],
    'Turks & Caicos': [2015],
    'Montserrat': [2015],
    'Bahamas': [2015],
    'Turkmenistan': [2015],
    'Pakistan': [2015],
    'Central African Republic': [2012],
    'Uganda': [2012],
    'Somalia': [2015],
    'Eritrea': [2015],
    'South Sudan': [2018],
    'Russia': [2021],
    'Honduras': [2021]
}


In [1586]:
latest_match_data['team 1 max value names'] = ''
latest_match_data['team 1 max values'] = ''
latest_match_data['team 1 max value scores'] = ''
latest_match_data['team 1 player in data that year'] = ''
latest_match_data['team 1 total value score'] = ''


latest_match_data['team 2 max values'] = ''
latest_match_data['team 2 max value names'] = ''

latest_match_data['team 2 max value scores'] = ''
latest_match_data['team 2 player in data that year'] = ''
latest_match_data['team 2 total value score'] = ''
latest_match_data['value_score_difference'] = ''

In [1614]:
i #start with index 9 on team 2 , index 3017

9

In [1631]:
# player, position, market_value_2, team_2_name, season_2, i, market_values, market_values_ii, team_2_max_value_names, teammates
#get_player_position(player, position, market_value_2, team_2_name, season_2, i, market_values, market_values_ii, team_2_max_value_names, teammates)

In [1637]:
latest_match_data.at[3419, 'Team.1.Players'] = "['', '', '', '', '', '', '', '', '', '', '']"

In [1643]:
#index got stopped on 3017

low_index = 5000 #4500
high_index = 5320 #4550
# low_index += 50
# high_index += 50
for index, row in latest_match_data.iterrows():
    if low_index <= index <= high_index:
        if row['Competition'] != 'Olympics':

            season = row['Season']
            team_1_name = row['Team.1.Name']
            team_2_name = row['Team.2.Name']

            # Check if the current team and season should be skipped
            if (team_1_name in skip_conditions and season in skip_conditions[team_1_name]) or \
               (team_2_name in skip_conditions and season in skip_conditions[team_2_name]):
                continue

            if latest_match_data.loc[index, 'Team.1.Players'] == [] or latest_match_data.loc[index, 'Team.2.Players'] == []:
                continue
    

            team_1_max_value_names = []
            team_2_max_value_names = []

            team_1_max_values = []
            team_2_max_values = []

            team_1_max_value_scores = []
            team_2_max_value_scores = []

            team_1_player_in_data_that_year = [] #for all the players in team 1's lineup, were they in the TM data that season. 
            team_2_player_in_data_that_year = [] #for all the players in team 2's lineup, were they in the TM data that season. 

            team_1_positions = eval(row['Team.1.Players_Positions'])
            team_2_positions = eval(row['Team.2.Players_Positions'])
            team_1_players = eval(row['Team.1.Players'])
            team_2_players = eval(row['Team.2.Players'])
            team_1_market_values = row['Team.1.Values.By.Player']
            team_2_market_values = row['Team.2.Values.By.Player']
            if team_1_name in africa_countries_list and season > 2023:
                season_1 = 2023
            else:
                season_1 = season
            if team_2_name in africa_countries_list and season > 2023:
                season_2 = 2023
            else:
                season_2 = season
            # Process team 1 players
            for i in range(0, 11):
                team_1_list_not_yet_processed = [x for x in team_1_players if x not in team_1_max_value_names]
                #print(f'{i}, the list of not yet processed is {team_1_list_not_yet_processed}')
                player = team_1_players[i]
                position = team_1_positions[i]
                market_value = eval(team_1_market_values)[i]
                teammates = [x for x in team_1_players if x != player]
                max_value_name, max_value, max_value_score, player_in_data_that_year = get_player_position(player, position, market_value, team_1_name, season_1, i, market_values, market_values_ii, team_1_max_value_names, teammates)
                team_1_max_value_names.append(max_value_name)
                team_1_max_values.append(max_value)
                team_1_max_value_scores.append(max_value_score / 100)
                team_1_player_in_data_that_year.append(player_in_data_that_year)

            # Process team 2 players
            for i in range(0, 11):
                team_2_list_not_yet_processed = [x for x in team_2_players if x not in team_2_max_value_names]
                player = team_2_players[i]
                position = team_2_positions[i]
                market_value_2 = eval(team_2_market_values)[i]
                teammates = [x for x in team_2_players if x != player]
                #print(f"player is {player}, list of teammmates i pass in is {teammates}")
                max_value_name, max_value, max_value_score, player_in_data_that_year = get_player_position(player, position, market_value_2, team_2_name, season_2, i, market_values, market_values_ii, team_2_max_value_names, teammates)
                team_2_max_value_names.append(max_value_name)
                team_2_max_values.append(max_value)
                team_2_max_value_scores.append(max_value_score / 100)
                team_2_player_in_data_that_year.append(player_in_data_that_year)
            
            latest_match_data.at[index, 'team 1 max value names'] = team_1_max_value_names
            latest_match_data.at[index, 'team 2 max value names'] = team_2_max_value_names
            latest_match_data.at[index, 'team 1 max values'] = team_1_max_values
            latest_match_data.at[index, 'team 2 max values'] = team_2_max_values
            latest_match_data.at[index, 'team 1 max value scores'] = team_1_max_value_scores
            latest_match_data.at[index, 'team 2 max value scores'] = team_2_max_value_scores
            latest_match_data.at[index, 'team 1 player in data that year'] = team_1_player_in_data_that_year
            latest_match_data.at[index, 'team 2 player in data that year'] = team_2_player_in_data_that_year
            latest_match_data.at[index, 'team 1 total value score'] = sum(team_1_max_value_scores)
            latest_match_data.at[index, 'team 2 total value score'] = sum(team_2_max_value_scores)
            score_difference = sum(team_1_max_value_scores) - sum(team_2_max_value_scores)
            latest_match_data.at[index, 'value_score_difference'] = score_difference
            #if score_difference
            #latest_match_data.at[index, 'value_score_diff_prediction_correct'] = 
            
        else:
            print(index, 'Olympics')

8 - no data with expanded search Tony Tsabedze Left Winger 120000 Eswatini 2018
max value score is greater than 100 Ahmed El Shenawy Goalkeeper 2160000 Mohamed El Shenawy 1700000.0 100
list of used players is  ['']
max value score is greater than 100 Karim Hafez Left-Back 1080000 Karim Hafez 675000.0 100
list of used players is  ['Mohamed El Shenawy', 'Ahmed Hegazy', 'Ayman Ashraf']
max value score is greater than 100 Virgil Vries Goalkeeper 375000 Maximilian Mbaeva 200000.0 100
list of used players is  ['']
max value score is greater than 100 Sadney Urikhob Centre-Forward 180000 Peter Shalulile 150000.0 100
list of used players is  ['Maximilian Mbaeva', 'Tiberius Lombard', 'Ananias Gebhardt', 'Riaan Hanamub', 'Denzil Haoseb', 'Ronald Ketjijere', 'Wangu Gome', 'Petrus Shitembi', 'Deon Hotto', 'Benson Shilongo']
7 - no data with expanded search Rainford Kalaba Left Winger 400000.0 Zambia 2018
max value score is greater than 100 Zainadine Centre-Back 1825000 Zainadine 1000000.0 100
list 

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_39501/4204884324.py:84: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  position_data_big = pd.concat([data_subset_position, data_subset_position_ii], ignore_index=True)


max value score is greater than 100 Jeffrey Schlupp Left Midfield 11400000 Jeffrey Schlupp 9000000.0 100
list of used players is  ['Richard Ofori', 'Mohammed Salisu', 'Alexander Djiku', 'Abdul Rahman Baba', 'Joseph Aidoo', 'Alfred Duncan', 'Iddrisu Baba']
max value score is greater than 100 Thulani Hlatshwayo Centre-Back 1000000.0 Taariq Fielies 900000.0 100
list of used players is  ['Ronwen Williams']
max value score is greater than 100 Andile Jali Defensive Midfield 880000 Njabulo Blom 625000.0 100
list of used players is  ['Ronwen Williams', 'Taariq Fielies', 'Mothobi Mvala', 'Aubrey Modiba']
max value score is greater than 100 Sifiso Hlanti Left-Back 1000000.0 Lyle Lakay 900000.0 100
list of used players is  ['Ronwen Williams', 'Taariq Fielies', 'Mothobi Mvala', 'Aubrey Modiba', 'Njabulo Blom', 'Thapelo Morena']
max value score is greater than 100 Rivaldo Coetzee Defensive Midfield 700000.0 Cafú Phete 600000.0 100
list of used players is  ['Ronwen Williams', 'Taariq Fielies', 'Moth

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_39501/4204884324.py:84: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  position_data_big = pd.concat([data_subset_position, data_subset_position_ii], ignore_index=True)


3 - no data with expanded search Mxolisi Simiso Manana Left-Back 0 Eswatini 2023
5 - no data with expanded search Frederic Ananou Right-Back 840000 Togo 2023
max value score is greater than 100 Leandro Andrade Attacking Midfield 540000 Patrick Andrade 500000.0 100
list of used players is  ['Bruno Varela', 'Logan Costa', 'Diney Borges', 'Telmo Arcanjo', 'Jovane', 'Kevin Lenini', 'Jamiro Monteiro']
8 - no data with expanded search Deon Hotto Left Winger 1250000 Namibia 2022
max value score is greater than 100 Cédric Amissi Defensive Midfield 640000 Jospin Nshimirimana 350000.0 100
list of used players is  ['Jonathan Nahimana', 'Frédéric Nsabiyumva', 'Youssouf Ndayishimiye', 'Marco Weymans', 'Philip Oslev']
max value score is greater than 100 Saidi Ntibazonkiza Attacking Midfield 675000 Irakoze Donasiyano 125000.0 100
list of used players is  ['Jonathan Nahimana', 'Frédéric Nsabiyumva', 'Youssouf Ndayishimiye', 'Marco Weymans', 'Philip Oslev', 'Jospin Nshimirimana', 'Gaël Bigirimana', 'Mo

...

max value score is greater than 100 Amor Layouni Right Winger 900000.0 Bassem Srarfi 500000.0 100
list of used players is  ['Bechir Ben Said', 'Montassar Talbi', 'Dylan Bronn', 'Ali Abdi', 'Yan Valery', 'Ellyes Skhiri', 'Aïssa Laïdouni', 'Hannibal', 'Mortadha Ben Ouanes', 'Wahbi Khazri']
max value score is greater than 100 Cafú Phete Defensive Midfield 720000 Njabulo Blom 475000.0 100
list of used players is  ['Ronwen Williams', 'Taariq Fielies', 'Mothobi Mvala']
max value score is greater than 100 Lyle Lakay Left-Back 1080000 Innocent Maela 1000000.0 100
list of used players is  ['Ronwen Williams', 'Taariq Fielies', 'Mothobi Mvala', 'Njabulo Blom', 'Teboho Mokoena', 'Yaya Sithole', 'Thapelo Morena']
max value score is greater than 100 Thembinkosi Lorch Left Winger 1110000 Luther Singh 800000.0 100
list of used players is  ['Ronwen Williams', 'Taariq Fielies', 'Mothobi Mvala', 'Njabulo Blom', 'Teboho Mokoena', 'Yaya Sithole', 'Thapelo Morena', 'Innocent Maela']
10 - no data with expand

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_39501/4204884324.py:84: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  position_data_big = pd.concat([data_subset_position, data_subset_position_ii], ignore_index=True)


10 - no data with expanded search Oscar Left-Back 5000000.0 Liberia 2023
max value score is greater than 100 Reinildo Mandava Left-Back 30000000.0 Reinildo Mandava 25000000.0 100
list of used players is  ['Ernan', 'Edmilson Dove', 'Mexer']
max value score is greater than 100 Edouard Mendy Goalkeeper 30000000.0 Edouard Mendy 12000000.0 100
list of used players is  ['']
max value score is greater than 100 Sessi D'Almeida Defensive Midfield 850000 Dokou Dodo 350000.0 100
list of used players is  ['Saturnin Allagbé', 'Cédric Hountondji', 'Olivier Verdon', 'David Kiki', 'Brandon Agounon']
max value score is greater than 100 Tidjani Anane Left Winger 360000 Yanick Aguemon 200000.0 100
list of used players is  ['Saturnin Allagbé', 'Cédric Hountondji', 'Olivier Verdon', 'David Kiki', 'Brandon Agounon', 'Dokou Dodo', 'Mattéo Ahlinvi']
max value score is greater than 100 Edouard Mendy Goalkeeper 30000000.0 Edouard Mendy 12000000.0 100
list of used players is  ['']
max value score is greater than

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_39501/4204884324.py:84: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  position_data_big = pd.concat([data_subset_position, data_subset_position_ii], ignore_index=True)


8 - no data with expanded search Fernando Jackson Left Midfield 0 Mauritius 2022
max value score is greater than 100 Noah Sonko Sundberg Centre-Back 1500000 James Gomez 450000.0 100
list of used players is  ['Modou Jobe', 'Omar Colley', 'Bubacarr Sanneh', 'Yahya Kalley']
max value score is greater than 100 Bubacarr Trawally Left Winger 960000 Alieu Fadera 500000.0 100
list of used players is  ['Modou Jobe', 'Omar Colley', 'Bubacarr Sanneh', 'Yahya Kalley', 'James Gomez', 'Ebrima Darboe', 'Hamza Barry', 'Yankuba Minteh', 'Musa Barrow', 'Ali Sowe']


In [1658]:
latest_match_data['value_score_difference'] = pd.to_numeric(latest_match_data['value_score_difference'], errors='coerce')
latest_match_data['Score.Difference'] = pd.to_numeric(latest_match_data['Score.Difference'], errors='coerce')
latest_match_data['value_score_diff_prediction_correct'] = np.where(
    ((latest_match_data['value_score_difference'] > 0) & (latest_match_data['Score.Difference'] > 0)) |
    ((latest_match_data['value_score_difference'] < 0) & (latest_match_data['Score.Difference'] < 0)),
    1,
    0
)

latest_match_data['value_score_diff_prediction_correct_inverse'] = np.where(
    ((latest_match_data['value_score_difference'] > 0) & (latest_match_data['Score.Difference'] < 0)) |
    ((latest_match_data['value_score_difference'] < 0) & (latest_match_data['Score.Difference'] > 0)),
    1,
    0
)

In [1652]:
latest_match_data['value_score_nan_boolean'] = np.where(pd.isna(latest_match_data['value_score_difference']), 1, 0)

In [1664]:
#convert to a CSV - DONE
latest_match_data.to_csv('latest_match_data_positions_fixed_home_away_FIXED_value_scores_ADDED_July_29.csv')

In [1128]:
#OLD VERSION OF SCRIPT 

low_index = 4501
high_index = 4501#4550
# low_index += 50
# high_index += 50
for index, row in latest_match_data.iterrows():
    if low_index <= index <= high_index:
        if row['Competition'] != 'Olympics':
            search_positions_expanded = ['']
            season = row['Season']
            team_1_name = row['Team.1.Name']
            team_2_name = row['Team.2.Name']


            team_1_positions = eval(row['Team.1.Players_Positions'])
            team_2_positions = eval(row['Team.2.Players_Positions'])
            team_1_players = eval(row['Team.1.Players'])
            team_2_players = eval(row['Team.2.Players'])
            team_1_market_values = row['Team.1.Values.By.Player']
            team_2_market_values = row['Team.2.Values.By.Player']
            for i in range(0, 11):
                0==0
                player = team_1_players[i]
                position = team_1_positions[i]
                market_value = eval(team_1_market_values)[i]

                search_positions = get_search_positions(position)

                #look for the player market value if it says zero in the row. 
                    #search market_values to see if it's nonzero in that season
                    #search market_values_ii to see if it's nonzero in that season

                if team_1_name == 'Türkiye':
                    if season > 2012:
                        team_1_name = 'Turkey'

                data_subset_position = market_values[
                    (market_values['Nationality'] == normalize_country_name(team_1_name)) &
                    (market_values['Season'] == season) &
                    (market_values['Position'].isin(search_positions))
                ]
                #['Market Value Numeric'].max()
                data_subset_position_ii = market_values_ii[
                    (market_values_ii['Nationality'] == normalize_country_name_ii(team_1_name)) &
                    (market_values_ii['Season'] == season) &
                    (market_values_ii['Position'].isin(search_positions))
                ]
                #if the data_subset_position is 0, expand the search
                if len(data_subset_position) == 0 and len(data_subset_position_ii) == 0:
                    #EXPAND THE SEARCH
                    if position == 'Central Midfield':
                        search_positions_expanded = ['Attacking Midfield', 'Defensive Midfield', 'Midfielder']
                    elif position == 'Attacking Midfield':
                        search_positions_expanded = ['Central Midfield', 'Midfielder']
                    elif position == 'Defensive Midfield':
                        search_positions_expanded = ['Central Midfield', 'Midfielder']

                    data_subset_position_expanded = market_values[
                        (market_values['Nationality'] == normalize_country_name(team_1_name)) &
                        (market_values['Season'] == season) &
                        (market_values['Position'].isin(search_positions_expanded))
                    ]

                    data_subset_position_expanded_ii = market_values_ii[
                        (market_values_ii['Nationality'] == normalize_country_name_ii(team_1_name)) &
                        (market_values_ii['Season'] == season) &
                        (market_values_ii['Position'].isin(search_positions_expanded))
                    ]

                    if len(data_subset_position_expanded) == 0 and len(data_subset_position_expanded_ii) == 0:
                        print(index, '- no data with expanded search', player, position, market_value, team_1_name, season)


            #looking for the highest value person at their position
                #search in market_values - yep
                #search in market_values_ii - yep
                #used the combined set from both - yep 

                #if the combined length is only 1 and it's the person, the answer is 1
                #if the combbined length is 0: - yep 
                    #expand the search - yep 
            for i in range(4, 5):
                0==0
                player = team_2_players[i]
                position = team_2_positions[i]
                market_value = eval(team_2_market_values)[i]

                search_positions = get_search_positions(position)

                #look for the player market value if it says zero in the row. 
                    #search market_values to see if it's nonzero in that season
                    #search market_values_ii to see if it's nonzero in that season

                if team_2_name == 'Türkiye':
                    if season > 2012:
                        team_2_name = 'Turkey'

                data_subset_position = market_values[
                    (market_values['Nationality'] == normalize_country_name(team_2_name)) &
                    (market_values['Season'] == season) &
                    (market_values['Position'].isin(search_positions))
                ]
                #['Market Value Numeric'].max()
                data_subset_position_ii = market_values_ii[
                    (market_values_ii['Nationality'] == normalize_country_name_ii(team_2_name)) &
                    (market_values_ii['Season'] == season) &
                    (market_values_ii['Position'].isin(search_positions))
                ]
                #if the data_subset_position is 0, expand the search
                if len(data_subset_position) == 0 and len(data_subset_position_ii) == 0:
                    #EXPAND THE SEARCH
                    if position == 'Central Midfield':
                        search_positions_expanded = ['Attacking Midfield', 'Defensive Midfield', 'Midfielder']
                    elif position == 'Attacking Midfield':
                        search_positions_expanded = ['Central Midfield', 'Midfielder']
                    elif position == 'Defensive Midfield':
                        search_positions_expanded = ['Central Midfield', 'Midfielder']

                    data_subset_position_expanded = market_values[
                        (market_values['Nationality'] == normalize_country_name(team_2_name)) &
                        (market_values['Season'] == season) &
                        (market_values['Position'].isin(search_positions_expanded))
                    ]

                    data_subset_position_expanded_ii = market_values_ii[
                        (market_values_ii['Nationality'] == normalize_country_name_ii(team_2_name)) &
                        (market_values_ii['Season'] == season) &
                        (market_values_ii['Position'].isin(search_positions_expanded))
                    ]

                    if len(data_subset_position_expanded) == 0 and len(data_subset_position_expanded_ii) == 0:
                        print(index, '- no data with expanded search', player, position, market_value, team_2_name, season)
        else:
            print(index, 'Olympics')                

                    

4501 - no data with expanded search Mustafa Elfadni Right Midfield 0 Sudan 2022


### Ad Hoc Fixes

In [1083]:
season_interest = 2021
nationality_interest = 'Honduras'

In [1085]:
latest_match_data[(latest_match_data['Season'] == 2021) & ((latest_match_data['Team.1.Name'] == 'Honduras') | (latest_match_data['Team.2.Name'] == 'Honduras'))]

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Match.ID,Competition,Match,Date,Season,Team.1.Name,Team.1.Code,Team.1.Players,Team.1.Players_Positions,Team.1.Values.By.Player,Team.2.Name,Team.2.Code,Team.2.Players,Team.2.Players_Positions,Team.2.Values.By.Player,Team.1.Confederation,Team.2.Confederation,Team.1.Score,Team.2.Score,score,Score.Difference,Team.1.Rank,Team.2.Rank,Rank.Difference,Market.Value.Difference,Team.1.Total.Market.Value,Team.2.Total.Market.Value,Team.1.Num.of.0.Value.Players,Team.2.Num.of.0.Value.Players,Number.of.Valuable.Players.Difference,Standardized_Rank_Difference,"Team 1, Tier 1 Global Superstars","Team 1, Tier 2 Global Superstars","Team 2, Tier 1 Global Superstars","Team 2, Tier 2 Global Superstars","Team 1, Tier 1 Confederation Stars","Team 1, Tier 2 Confederation Stars","Team 2, Tier 1 Confederation Stars","Team 2, Tier 2 Confederation Stars",mv_difference_abs,MV_Prediction_Correct,smaller_team_value,rank_difference_abs,rank_prediction_correct,smaller_team_rank,"Team 1, Total Global Superstars","Team 2, Total Global Superstars",Global_Superstar_difference,Global_Superstar_difference_abs,Global_Superstar_difference_prediction_correct,Global_Superstar_difference_prediction_correct_inverse,Number_of_valuable_players_prediction_correct,Number_of_valuable_players_prediction_correct_inverse,less_valuable_team_FIFA_ranking,worse_ranked_team_value,"Team 1, Total Confederation Stars","Team 2, Total Confederation Stars",Confederation_star_difference,Confederation_difference_abs,MV_Prediction_Correct_inverse,Rank_Prediction_Correct_inverse,Team with Superstar MV,Team without Superstar MV,Confederation_star_prediction_correct,Confederation_star_prediction_correct_inverse,Team.1.Wins,Team.2.Wins,Smaller Number of 0 Value Players,Number.of.Valuable.Players.Difference.Absolute
88,88,88,88,Olympics_30,Olympics,Honduras vs Romania,2021-07-22,2021,Honduras,HN,"['Jorge Álvarez', 'Denil Maldonado', 'Wesly Decas', 'Edwin Rodríguez', 'Alex Güity', 'José García', 'Elvin Oliva', 'Carlos Pineda', 'Rigoberto Rivas', 'Jorge Benguché', 'Douglas Martínez']",NaN,"225000,5e+05,150000,2e+05,0,0,0,0,725000,220000,420000",Romania,RO,"['Marian Aioani', 'Andrei Rațiu', 'Virgil Ghiță', 'Marius Marin', 'Alex Dobre', 'Florin Ștefan', 'Marco Dulca', 'Valentin Gheorghe', 'Andrei Virgil Ciobanu', 'George Dănuț Ganea', 'Alexandru Ștefan Pașcanu']",NaN,"450000,3e+05,1300000,5e+05,1200000,1200000,650000,9e+05,1040000,320000,750000",CONCACAF,UEFA,0,1,0:1 NA,-1,67,43,-24,-6170000.0,2440000.0,8610000.0,4,0,-4,-0.388460,0,0,0,0,0,0,0,0,6170000.0,1,2440000.0,24,1,67,0,0,0,0,0,0,1,0,67.0,2440000.0,0,0,0,0,0,0,8610000.0,2440000.0,1,0,0,1,0,4
89,89,89,89,Olympics_43,Olympics,New Zealand vs Honduras,2021-07-25,2021,New Zealand,NZ,"['Michael Woud', 'Winston Reid', 'Joe Bell', 'Nando Pijnaker', 'Dane Ingham', 'Clayton Lewis', 'Liberato Cacace', 'Elijah Just', 'Chris Wood', 'Callum McCowatt', 'Gianni Stensness']",NaN,"450000,6e+05,1500000,150000,450000,450000,2500000,350000,1e+07,350000,325000",Honduras,HN,"['Denil Maldonado', 'Wesly Decas', 'Jorge Álvarez', 'Edwin Rodríguez', 'Luis Palma', 'Alex Güity', 'Cristopher Meléndez', 'José García', 'Jorge Benguché', 'Douglas Martínez', 'Alejandro Reyes']",NaN,"5e+05,150000,225000,2e+05,125000,0,0,0,220000,420000,150000",OFC,CONCACAF,2,3,2:3 NA,-1,122,67,-55,15135000.0,17125000.0,1990000.0,0,3,3,-0.879403,0,0,0,0,3,0,0,0,15135000.0,0,1990000.0,55,1,122,0,0,0,0,0,0,0,1,67.0,17125000.0,3,0,3,3,1,0,1990000.0,1990000.0,0,0,0,1,0,3
92,92,92,92,Olympics_57,Olympics,South Korea vs Honduras,2021-07-28,2021,South Korea,KR,"['Bum-keun Song', 'Jin-gyu Kim', 'Ui-jo Hwang', 'Chang-hoon Kwon', 'Du-jae Won', 'Ji-su Park', 'Young-woo Seol', 'Dong-jun Lee', 'Tae-wook Jeong', 'Kim Jin-ya', 'Yun-sung Kang']",NaN,"8e+05,325000,2500000,2800000,950000,7e+05,960000,650000,960000,550000,375000",Honduras,HN,"['Denil Maldonado', 'Wesly Decas', 'Luis Palma', 'Edwin Rodríguez', 'Alex Güity', 'Jonathan Núñez', 'Elvin Oliva', 'Carlos Pineda', 'Carlos

In [1078]:
market_values[market_values['Name'] == 'Brayan Moya']#Brayan Moya

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Name,Nationality,Position,Market Value,Season,Team,Birthday,Age,JerseyNumber,PlayerID,Accents Removed,Team 1 Code,Club,Market Value Numeric,Confederation
36825,36825,36825,36825,Brayan Moya,Honduras,Attacking Midfield,€325k,2019,Zulia FC,"Oct 19, 1992",26,13,"Brayan Moya_Oct 19, 1992_Honduras",Brayan Moya,HN,NaN,325000.0,CONCACAF
36896,36896,36896,36896,Brayan Moya,Honduras,Attacking Midfield,€400k,2020,Without Club,"Oct 19, 1992",27,-,"Brayan Moya_Oct 19, 1992_Honduras",Brayan Moya,HN,NaN,400000.0,CONCACAF
36998,36998,36998,36998,Brayan Moya,Honduras,Attacking Midfield,€275k,2022,CD Olimpia,"Oct 19, 1992",29,-,"Brayan Moya_Oct 19, 1992_Honduras",Brayan Moya,HN,NaN,275000.0,CONCACAF


In [1081]:
market_values[(market_values['Nationality'] == nationality_interest) & (market_values['Season'] == season_interest)]#['Position'].unique()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Name,Nationality,Position,Market Value,Season,Team,Birthday,Age,JerseyNumber,PlayerID,Accents Removed,Team 1 Code,Club,Market Value Numeric,Confederation
36927,36927,36927,36927,Harold Fonseca,Honduras,Goalkeeper,€225k,2021,Marathón,"Oct 8, 1993",27,1,"Harold Fonseca_Oct 8, 1993_Honduras",Harold Fonseca,HN,NaN,225000.0,CONCACAF
36928,36928,36928,36928,Edrick Menjívar,Honduras,Goalkeeper,€175k,2021,CD Olimpia,"Mar 1, 1993",28,1,"Edrick Menjívar_Mar 1, 1993_Honduras",Edrick Menjivar,HN,NaN,175000.0,CONCACAF
36929,36929,36929,36929,Luis López,Honduras,Goalkeeper,€175k,2021,Real España,"Sep 13, 1993",27,22,"Luis López_Sep 13, 1993_Honduras",Luis Lopez,HN,NaN,175000.0,CONCACAF
36930,36930,36930,36930,Marlon Licona,Honduras,Goalkeeper,€125k,2021,FC Motagua,"Feb 9, 1991",30,22,"Marlon Licona_Feb 9, 1991_Honduras",Marlon Licona,HN,NaN,125000.0,CONCACAF
36931,36931,36931,36931,Denil Maldonado,Honduras,Centre-Back,€500k,2021,CD Everton,"May 26, 1998",23,15,"Denil Maldonado_May 26, 1998_Honduras",Denil Maldonado,HN,NaN,500000.0,CONCACAF
36932,36932,36932,36932,Marcelo Pereira,Honduras,Centre-Back,€275k,2021,FC Motagua,"May 27, 1995",26,4,"Marcelo Pereira_May 27, 1995_Honduras",Marcelo Pereira,HN,NaN,275000.0,CONCACAF
36933,36933,36933,36933,Marcelo Santos,Honduras,Centre-Back,€125k,2021,FC Motagua,"Aug 2, 1992",28,3,"Marcelo Santos_Aug 2, 1992_Honduras",Marcelo Santos,HN,NaN,125000.0,CONCACAF
36934,36934,36934,36934,Devron García,Honduras,Centre-Back,-,2021,Real España,"Feb 17, 1996",25,2,"Devron García_Feb 17, 1996_Honduras",Devron Garcia,HN,NaN,0.0,CONCACAF
36935,36935,36935,36935,Junior Garcia,Honduras,Centre-Back,-,2021,Real España,"Oct 7, 2001",19,-,"Junior Garcia_Oct 7, 2001_Honduras",Junior Garcia,HN,NaN,0.0,CONCACAF
36936,36936,36936,36936,Allans Vargas,Honduras,Centre-Back,-,2021,No Transfer History,"Sep 25, 1993",27,2,"Allans Vargas_Sep 25, 1993_Honduras",Allans Vargas,HN,NaN,0.0,CONCACAF


In [1082]:
market_values_ii[(market_values_ii['Nationality'] == nationality_interest) & (market_values_ii['Season'] == season_interest)]#['Position'].unique()

,Unnamed: 0,Name,Nationality,Position,Market Value,Season,League,Team,Team 1 Code,Birthday,Age,PlayerID,Accents Removed,Market Value Numeric
14789,14789,Alberth Elis,Honduras,Right Winger,6500000.0,2021,Ligue 1,Bordeaux,HN,"Feb 12, 1996",26.0,"Alberth Elis_Feb 12, 1996_Honduras",Alberth Elis,6500000.0
31499,31499,Choco Lozano,Honduras,Centre-Forward,4000000.0,2021,La Liga,Cádiz,HN,"Apr 25, 1993",29.0,"Choco Lozano_Apr 25, 1993_Honduras",Choco Lozano,4000000.0
67048,67048,Maynor Figueroa,Honduras,Left-Back,200000.0,2021,NaN,Retired,HN,"May 02, 1983",37.0,"Maynor Figueroa_May 02, 1983_Honduras",Maynor Figueroa,200000.0
67067,67067,Bryan Róchez,Honduras,Centre-Forward,500000.0,2021,NaN,União de Leiria,HN,"Jan 01, 1995",25.0,"Bryan Róchez_Jan 01, 1995_Honduras",Bryan Rochez,500000.0
67075,67075,Rubilio Castillo,Honduras,Centre-Forward,200000.0,2021,NaN,CD Motagua Tegucigalpa,HN,"Nov 26, 1991",29.0,"Rubilio Castillo_Nov 26, 1991_Honduras",Rubilio Castillo,200000.0
67083,67083,Wesly Decas,Honduras,Left-Back,150000.0,2021,NaN,CD Motagua Tegucigalpa,HN,"Aug 11, 1999",21.0,"Wesly Decas_Aug 11, 1999_Honduras",Wesly Decas,150000.0
88508,88508,Luis López,Honduras,Goalkeeper,175000.0,2021,NaN,Real CD España,HN,"Sep 13, 1993",27.0,"Luis López_Sep 13, 1993_Honduras",Luis Lopez,175000.0
88623,88623,Jerry Bengtson,Honduras,Centre-Forward,275000.0,2021,NaN,CD Olimpia,HN,"Apr 08, 1987",33.0,"Jerry Bengtson_Apr 08, 1987_Honduras",Jerry Bengtson,275000.0
88632,88632,Romell Quioto,Honduras,Centre-Forward,2000000.0,2021,NaN,Tractor FC,HN,"Aug 09, 1991",29.0,"Romell Quioto_Aug 09, 1991_Honduras",Romell Quioto,2000000.0
88685,88685,Eddie Hernández,Honduras,Centre-Forward,325000.0,2021,NaN,Mohammedan SC (Kolkata),HN,"Feb 27, 1991",29.0,"Eddie Hernández_Feb 27, 1991_Honduras",Eddie Hernandez,325000.0


In [1068]:
market_values[market_values['Nationality'] == nationality_interest]['Season'].value_counts()

Season
2020    85
2011    65
2016    56
2022    55
2015    53
2014    48
2013    47
2019    45
2012    42
2018    37
2021    37
2017    35
2023    23
Name: count, dtype: int64

In [1069]:
market_values_ii[market_values_ii['Nationality'] == nationality_interest]['Season'].value_counts()

Season
2020    85
2016    57
2022    55
2015    53
2014    50
2013    47
2019    46
2021    39
2018    39
2017    35
2023    25
Name: count, dtype: int64

In [807]:
set(market_values['Nationality'].unique())

{'Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bosnia-Herzegovina',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Chinese Taipei',
 'Colombia',
 'Comoros',
 'Cook Islands',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Curaçao',
 'Cyprus',
 'Czech Republic',
 'Democratic Republic of the Congo',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'England',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Faroe Islands',
 'Fiji',
 'Finland',
 'Franc

In [808]:
set(market_values_ii['Nationality'].unique())

{'Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bosnia-Herzegovina',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Chinese Taipei',
 'Colombia',
 'Comoros',
 'Congo',
 'Cook Islands',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Curacao',
 'Curaçao',
 'Cyprus',
 'Czech Republic',
 'DR Congo',
 'Democratic Republic of the Congo',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'England',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethi